## Features:
 - outs, on_1b, on_2b, on_3b, launch_speed, launch_angle, spray_angle, batter_speed, fielder_positioning, OAA_for_batter_handedness (for current game yr, standardized)

In [36]:
import requests
import re
import json
import pandas as pd
import numpy as np
from datetime import datetime

In [71]:
# first scrape OAA
url = f"https://baseballsavant.mlb.com/leaderboard/outs_above_average?type=Fielding_Team&startYear=2021&endYear={datetime.now().year}&split=yes&team=&range=year&min=q&pos=of&roles=&viz=hide&sort=5&sortDir=desc"

response = requests.get(url)
data = re.search(r"data = (.*);", response.text).group(1)
data = json.loads(data)
df = pd.DataFrame(data)

df = df[['year', 'entity_id', 'entity_name', 'outs_above_average_rhh', 'outs_above_average_lhh']].sort_values(by=['entity_id', 'year'])

means = df.groupby('year')[['outs_above_average_rhh', 'outs_above_average_lhh']].mean()
stds = df.groupby('year')[['outs_above_average_rhh', 'outs_above_average_lhh']].std()

for col in ('outs_above_average_rhh', 'outs_above_average_lhh'):
    df[f'{col}_standardized'] = df.apply(lambda row: (row[col] - means.loc[row['year']][col]) / stds.loc[row['year']][col], axis=1)




,year,entity_id,entity_name,outs_above_average_rhh,outs_above_average_lhh,outs_above_average_rhh_standardized,outs_above_average_lhh_standardized
3,2021,108,Angels,-8,-6,-0.986031,-1.104716
0,2022,108,Angels,2,6,0.088866,0.995313
1,2023,108,Angels,0,4,-0.191144,0.873324
2,2024,108,Angels,0,3,-0.052330,0.907826
35,2021,109,D-backs,3,-4,0.300096,-0.710175
...,...,...,...,...,...,...,...
117,2024,147,Yankees,1,2,0.261650,0.618094
21,2021,158,Brewers,10,2,1.118541,0.473450
22,2022,158,Brewers,6,3,0.515420,0.572775
20,2023,158,Brewers,16,8,2.288558,1.540833


In [57]:
df.columns


In [65]:
means.loc['2023']

outs_above_average_rhh    1.233333
outs_above_average_lhh   -1.233333
Name: 2023, dtype: float64

In [70]:



df.set_index('year').loc['2024'].sort_values(by='outs_above_average_lhh_standardized', ascending=False)

,entity_id,entity_name,outs_above_average_rhh,outs_above_average_lhh,outs_above_average_rhh_standardized,outs_above_average_lhh_standardized
year,,,,,,
2024,141,Blue Jays,4,8,1.203591,2.356484
2024,138,Cardinals,0,4,-0.052330,1.197557
2024,109,D-backs,4,4,1.203591,1.197557
2024,108,Angels,0,3,-0.052330,0.907826
2024,117,Astros,4,3,1.203591,0.907826
2024,116,Tigers,4,3,1.203591,0.907826
2024,147,Yankees,1,2,0.261650,0.618094
2024,137,Giants,-4,2,-1.308251,0.618094
2024,119,Dodgers,4,2,1.203591,0.618094


,year,entity_id,entity_name,outs_above_average_rhh,outs_above_average_lhh
3,2021,108,Angels,-8,-6
0,2022,108,Angels,2,6
1,2023,108,Angels,0,4
2,2024,108,Angels,0,3
35,2021,109,D-backs,3,-4
...,...,...,...,...,...
117,2024,147,Yankees,1,2
21,2021,158,Brewers,10,2
22,2022,158,Brewers,6,3
20,2023,158,Brewers,16,8


In [ ]:
# second scrape batter speed
url = "https://baseballsavant.mlb.com/leaderboard/sprint_speed?min_season=2024&max_season=2024&position=&team=&min=5"